# Data Cleansing

In [1]:
import pandas as pd
import json
import seaborn as sns
import tqdm
import re

In [2]:
vcdb = pd.read_csv('data/vcdb.csv')

C:\Users\josep\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,116,163,185,217,264,850,851,852,853,1433,1722,1753,1893,2157,2159,2383) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
[col for col in vcdb if 'year' in col]

['plus.attribute.confidentiality.credit_monitoring_years',
 'plus.dbir_year',
 'plus.timeline.notification.year',
 'timeline.incident.year']

In [3]:
with open('data/vcdb-enum.json') as f:
    enums = json.load(f)
    
with open('data/vcdb-merged.json') as f:
    schema = json.load(f)

In [4]:
keynames = []
with open('data/vcdb-keynames-real.txt') as f:
    for line in f:
        keynames.append(line.replace('\n', ''))
        

In [7]:
pd.Series(list(enums.keys())).to_clipboard()

In [8]:
features = set([col[:col.rindex('.')]+'.' if col.count('.') > 1 else col for col in vcdb])

In [9]:
[feat for feat in features if feat.endswith('.')]

['victim.revenue.iso_currency_code.',
 'plus.event_chain.action.',
 'impact.loss.rating.',
 'attribute.confidentiality.state.',
 'action.physical.',
 'action.physical.vector.',
 'asset.hosting.',
 'asset.management.',
 'plus.pci.',
 'plus.pci.req_10.',
 'plus.asset.',
 'action.hacking.variety.',
 'plus.pci.req_4.',
 'actor.external.variety.',
 'plus.pci.req_2.',
 'impact.loss.variety.',
 'timeline.containment.unit.',
 'plus.pci.req_11.',
 'plus.pci.req_3.',
 'plus.timeline.notification.',
 'asset.assets.variety.',
 'attribute.confidentiality.data.amount.',
 'plus.pci.compliance_status.',
 'attribute.confidentiality.data_disclosure.',
 'action.environmental.variety.',
 'plus.event_chain.actor.',
 'action.misuse.variety.',
 'plus.event_chain.',
 'plus.antiforensic_measures.',
 'victim.orgsize.',
 'victim.revenue.',
 'victim.industry2.',
 'asset.assets.amount.',
 'asset.country.',
 'plus.asset_os.',
 'plus.pci.req_1.',
 'action.misuse.vector.',
 'asset.governance.',
 'plus.attack_difficul

# Cleaning Functions

In [10]:
def decode(row_subset):
    if row_subset.sum() > 1:
        raise ValueError('Multiple Options Selected')
    for col in row_subset.index:
        if row_subset[col]:
            return col

In [11]:
def decode_feature(vcdb, pattern, cname='', split=True, drop=True):
    ohe_cols = [col for col in vcdb.columns if col.startswith(pattern)]
    if cname == '':
        cname = pattern[:pattern.rindex('.')]
    decoded = vcdb[ohe_cols].apply(decode, axis=1)
    if split:
        decoded = (
            decoded
                .str.split('.')
                .str[-1]
        )        
    if drop:
        vcdb = vcdb.drop(labels=ohe_cols, axis=1)
        if len([col for col in vcdb if col.startswith(pattern)]) != 0:
            raise Exception('Bad pattern supplied')
    
    vcdb[cname] = decoded
    return vcdb

# Incident Tracking Variables

In [12]:
tracking_cols = [
    'incident_id',
    'source_id'
]

## Target Variables

### Impact

In [13]:
enums['impact'].keys()

dict_keys(['loss', 'overall_rating', 'iso_currency_code'])

#### Security Incident

In [14]:
vcdb = vcdb.pipe(decode_feature, pattern='security_incident.')
vcdb['security_incident'].value_counts()

Confirmed    7759
Suspected      67
Near miss       6
Name: security_incident, dtype: int64

#### Impact Overall rating

In [16]:
vcdb = vcdb.pipe(decode_feature, pattern='impact.overall_rating.')
vcdb['impact.overall_rating'].value_counts(dropna=False)

Unknown          5763
NaN              2043
Distracting        13
Damaging            8
Painful             5
Insignificant       1
Name: impact.overall_rating, dtype: int64

#### Impact overall_amount, overall_min_amount, overall_max_amount

In [17]:
vcdb['impact.overall_amount'].value_counts(dropna=False)

NaN          7753
1000000.0       6
60000.0         5
100000.0        3
50000.0         3
             ... 
330000.0        1
13000.0         1
3000000.0       1
77777.0         1
325000.0        1
Name: impact.overall_amount, Length: 63, dtype: int64

In [19]:
vcdb['impact.overall_max_amount'].value_counts(dropna=False)

NaN       7832
1500.0       1
Name: impact.overall_max_amount, dtype: int64

In [20]:
vcdb['impact.overall_min_amount'].value_counts(dropna=False)

NaN          7831
3700000.0       1
300.0           1
Name: impact.overall_min_amount, dtype: int64

#### Impact Currency Code

Most of these columns are false and therefore not of use to us

In [21]:
vcdb = vcdb.pipe(decode_feature, pattern='impact.iso_currency_code.')
vcdb['impact.iso_currency_code'].value_counts(dropna=False)

NaN    7636
USD     153
GBP      28
EUR       8
THB       2
CZK       1
INR       1
CAD       1
AUD       1
ZAR       1
KRW       1
Name: impact.iso_currency_code, dtype: int64

### Impact Loss


In [22]:
[col for col in vcdb if 'impact.loss.' in col]

['impact.loss.amount',
 'impact.loss.max_amount',
 'impact.loss.min_amount',
 'impact.loss.rating.Major',
 'impact.loss.rating.Minor',
 'impact.loss.rating.Moderate',
 'impact.loss.rating.None',
 'impact.loss.rating.Unknown',
 'impact.loss.variety.Asset and fraud',
 'impact.loss.variety.Brand damage',
 'impact.loss.variety.Business disruption',
 'impact.loss.variety.Competitive advantage',
 'impact.loss.variety.Legal and regulatory',
 'impact.loss.variety.Operating costs',
 'impact.loss.variety.Other',
 'impact.loss.variety.Response and recovery']

#### Impact Loss Rating

All of these columns appear to be False and therefore quite useless to us

In [23]:
vcdb[[col for col in vcdb if 'impact.loss.rating.' in col]]
assert not vcdb[[col for col in vcdb if 'impact.loss.rating.' in col]].any().any()
vcdb = vcdb.drop(labels=[col for col in vcdb if 'impact.loss.rating.' in col], axis=1)

#### Impact Loss Variety

All of these columns appear to be False and therefore quite useless to us

In [24]:
vcdb[[col for col in vcdb if 'impact.loss.variety.' in col]]
assert not vcdb[[col for col in vcdb if 'impact.loss.variety.' in col]].any().any()

In [25]:
vcdb= vcdb.pipe(decode_feature, pattern='impact.loss.variety.')
vcdb['impact.loss.variety'].value_counts(dropna=False)

NaN    7833
Name: impact.loss.variety, dtype: int64

In [26]:
assert vcdb['impact.loss.max_amount'].isnull().all()
assert vcdb['impact.loss.min_amount'].isnull().all()
assert vcdb['impact.loss.variety'].isnull().all()
vcdb = vcdb.drop(labels=['impact.loss.max_amount', 'impact.loss.min_amount', 'impact.loss.variety'], axis=1)

### Impact Notes

In [27]:
vcdb['impact.notes'].notnull().value_counts()

False    7700
True      133
Name: impact.notes, dtype: int64

In [28]:
vcdb.loc[vcdb['impact.notes'].notnull(), 'impact.notes'].sample(5)

5304    The damage affected servers that stored person...
4298       "making transactions worth crores of rupees ";
3868     fraud losses totaled $63M over course of scheme;
4100                                                Fine;
5732    Lost business partnership with StartPage;Delay...
Name: impact.notes, dtype: object

In [29]:
enums['confidence']

['High', 'Medium', 'Low', 'None']

In [30]:
vcdb = vcdb.pipe(decode_feature, pattern='confidence.')
vcdb['confidence'].value_counts(dropna=False)

NaN       7158
High       425
Medium     181
Low         68
None         1
Name: confidence, dtype: int64

### Incident Timeline
The timeline of events leading up to and following an incident varies greatly depending on a multitude of factors. VERIS tracks the following incident milestones, not all of which are applicable to every incident:

1. First malicious action: Beginning of the threat actor's malicious actions against the victim. Port scans, initiating a brute-force attack, and even physical recon, are a few examples. This is only relevant to intentional and malicious actions.
2. Initial compromise: First point at which a security attribute (C/P, I/A, A/U) of an information asset was compromised.
3. Data exfiltration: First point at which non-public data was taken from the victim environment. Only applicable to data compromise events.
4. Incident discovery: When the organization first learned the incident had occurred.
5. Containment/restoration: Point at which the incident is contained (e.g., the “bleeding is stopped”) or restored (e.g., fully functional)”.

While this may be useful to capture some descriptive statistics on different incidents, this will not likely be useful to us in doing predictive analytics.

In [31]:
enums['timeline']

{'compromise': {'unit': ['Seconds',
   'Minutes',
   'Hours',
   'Days',
   'Weeks',
   'Months',
   'Years',
   'Never',
   'NA',
   'Unknown']},
 'exfiltration': {'unit': ['Seconds',
   'Minutes',
   'Hours',
   'Days',
   'Weeks',
   'Months',
   'Years',
   'Never',
   'NA',
   'Unknown']},
 'discovery': {'unit': ['Seconds',
   'Minutes',
   'Hours',
   'Days',
   'Weeks',
   'Months',
   'Years',
   'Never',
   'NA',
   'Unknown']},
 'containment': {'unit': ['Seconds',
   'Minutes',
   'Hours',
   'Days',
   'Weeks',
   'Months',
   'Years',
   'Never',
   'NA',
   'Unknown']}}

In [32]:
timeline_cols = [col for col in vcdb if col.startswith('timeline.')]
vcdb = vcdb.drop(labels=timeline_cols, axis=1)

In [33]:
time_keywords = ['day', 'year', 'month']
time_cols = [col for col in vcdb if any([w in col for w in time_keywords])]
time_cols += ['plus.created', 'plus.modified']
time_cols

['plus.attribute.confidentiality.credit_monitoring_years',
 'plus.dbir_year',
 'plus.timeline.notification.day',
 'plus.timeline.notification.month',
 'plus.timeline.notification.year',
 'plus.created',
 'plus.modified']

In [34]:
vcdb = vcdb.drop(labels=time_cols, axis=1)

### Victim related features

In [35]:
enums['victim'].keys()

dict_keys(['employee_count', 'country', 'revenue'])

In [36]:
keys = {}
for k, v in enums['victim'].items():
    if type(v) == dict:
        keys[k] = v.keys()
    elif type(v) == list:
        keys[k] = None

In [37]:
keys

{'employee_count': None,
 'country': None,
 'revenue': dict_keys(['iso_currency_code'])}

#### Victim Employee Count

Check that these employee categories are mutually exclusive

In [38]:
victim_employee_count_cols = [col for col in vcdb if 'victim.employee_count.' in col]

# This assertion performs the check
assert (vcdb[victim_employee_count_cols].sum(axis=1) == 1).all()
vcdb[victim_employee_count_cols].head()

,victim.employee_count.1 to 10,victim.employee_count.10001 to 25000,victim.employee_count.1001 to 10000,victim.employee_count.101 to 1000,victim.employee_count.11 to 100,victim.employee_count.25001 to 50000,victim.employee_count.50001 to 100000,victim.employee_count.Large,victim.employee_count.Over 100000,victim.employee_count.Small,victim.employee_count.Unknown
0,False,False,False,False,False,False,False,False,True,False,False
1,False,False,False,True,False,False,False,False,False,False,False
2,False,False,True,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,True
4,False,False,False,False,False,False,False,False,True,False,False


In [39]:
vcdb = vcdb.pipe(decode_feature, pattern='victim.employee_count.')
vcdb['victim.employee_count'].value_counts()

Unknown            2625
1001 to 10000      1058
Over 100000        1002
101 to 1000         835
11 to 100           675
1 to 10             414
10001 to 25000      374
Small               340
Large               234
25001 to 50000      169
50001 to 100000     107
Name: victim.employee_count, dtype: int64

#### Victim Country

In [40]:
victim_country_cols = [col for col in vcdb if 'victim.country' in col]

# This assertion performs the check
assert (vcdb[victim_country_cols].sum(axis=1) == 1).all()
vcdb[victim_country_cols].head()

,victim.country.AD,victim.country.AE,victim.country.AF,victim.country.AG,victim.country.AI,victim.country.AL,victim.country.AM,victim.country.AO,victim.country.AQ,victim.country.AR,...,victim.country.VI,victim.country.VN,victim.country.VU,victim.country.WF,victim.country.WS,victim.country.YE,victim.country.YT,victim.country.ZA,victim.country.ZM,victim.country.ZW
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [41]:
vcdb = vcdb.pipe(decode_feature, pattern='victim.country.')
vcdb['victim.country'].value_counts(dropna=False)

US         5700
GB          492
CA          306
Unknown     191
AU          121
           ... 
NP            1
MT            1
TZ            1
TJ            1
ZM            1
Name: victim.country, Length: 136, dtype: int64

#### Victim Revenue

In [42]:
vcdb['victim.revenue.amount'].isnull().value_counts()

True     7328
False     505
Name: victim.revenue.amount, dtype: int64

In [43]:
victim_revenue_currency_cols = [col for col in vcdb if 'victim.revenue.iso_currency_code.' in col]

# We find that most of the victim revenue currencies are null
print('%s not null values found'%(vcdb[victim_revenue_currency_cols].sum(axis=1) == 1).sum())

vcdb[victim_revenue_currency_cols].head()

513 not null values found


,victim.revenue.iso_currency_code.AED,victim.revenue.iso_currency_code.AFN,victim.revenue.iso_currency_code.ALL,victim.revenue.iso_currency_code.AMD,victim.revenue.iso_currency_code.ANG,victim.revenue.iso_currency_code.AOA,victim.revenue.iso_currency_code.ARS,victim.revenue.iso_currency_code.AUD,victim.revenue.iso_currency_code.AWG,victim.revenue.iso_currency_code.AZN,...,victim.revenue.iso_currency_code.WST,victim.revenue.iso_currency_code.XAF,victim.revenue.iso_currency_code.XCD,victim.revenue.iso_currency_code.XDR,victim.revenue.iso_currency_code.XOF,victim.revenue.iso_currency_code.XPF,victim.revenue.iso_currency_code.YER,victim.revenue.iso_currency_code.ZAR,victim.revenue.iso_currency_code.ZMK,victim.revenue.iso_currency_code.ZWD
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [44]:
vcdb = vcdb.pipe(decode_feature, pattern='victim.revenue.iso_currency_code.')
vcdb['victim.revenue.iso_currency_code'].value_counts(dropna=False)

NaN    7320
USD     510
GBP       3
Name: victim.revenue.iso_currency_code, dtype: int64

### Victim Orgsize

In [45]:
vcdb[[col for col in vcdb if col.startswith('victim.orgsize.')]].sum(axis=1).value_counts()

1    5208
0    2625
dtype: int64

In [46]:
vcdb = vcdb.pipe(decode_feature, pattern='victim.orgsize.')
vcdb['victim.orgsize'].value_counts(dropna=False)

Large    2944
NaN      2625
Small    2264
Name: victim.orgsize, dtype: int64

### Victim Industry

In [47]:
vcdb['victim.industry']

0       923140
1       621111
2       622110
3        51919
4       923140
         ...  
7828    622110
7829    524210
7830    541990
7831       622
7832    923140
Name: victim.industry, Length: 7833, dtype: object

### Victim Industry2

In [59]:
vcdb['victim.industry.name'].unique()

array(['Public ', 'Healthcare ', 'Information ', 'Utilities ',
       'Other Services ', 'Finance ', 'Educational ', 'Retail ',
       'Manufacturing ', 'Transportation ', 'Professional ',
       'Accomodation ', 'Administrative ', 'Unknown', 'Trade ',
       'Entertainment ', 'Real Estate ', 'Management ', 'Construction ',
       'Mining ', 'Agriculture '], dtype=object)

In [48]:
(vcdb['victim.industry2'].astype(int) == (
    vcdb[[col for col in vcdb if col.startswith('victim.industry2.')]]
        .apply(decode, axis=1)
        .str.split('.')
        .str[-1]
        .astype(int))).all()

True

In [49]:
vcdb = vcdb.drop(labels=[col for col in vcdb if col.startswith('victim.industry2.')], axis=1)

## Check Results

In [60]:
[col for col in vcdb if col.startswith('victim.')]

['victim.industry',
 'victim.locations_affected',
 'victim.notes',
 'victim.region',
 'victim.revenue.amount',
 'victim.secondary.amount',
 'victim.secondary.notes',
 'victim.secondary.victim_id',
 'victim.state',
 'victim.victim_id',
 'victim.industry2',
 'victim.industry3',
 'victim.industry.name',
 'victim.employee_count',
 'victim.country',
 'victim.revenue.iso_currency_code',
 'victim.orgsize']

## Action related features

For these action related features we may wish to keep them one-hot encoded since an incident can be caused by multiple actions for predictions at incident level.

For predictions at action level down the road we can generate more training samples by melting incidents with more than one action.

In [51]:
enums['action'].keys()

dict_keys(['hacking', 'malware', 'social', 'error', 'misuse', 'physical', 'environmental', 'unknown'])

The features and their respective one-hot encoded values are of similar depth in the schema, meaning we can write a simple for loop in order to turn these into categorical variables

In [52]:
{k: v.keys() for k, v in enums['action'].items()}

{'hacking': dict_keys(['variety', 'vector', 'result']),
 'malware': dict_keys(['variety', 'vector', 'result']),
 'social': dict_keys(['variety', 'vector', 'target', 'result']),
 'error': dict_keys(['variety', 'vector']),
 'misuse': dict_keys(['variety', 'vector', 'result']),
 'physical': dict_keys(['variety', 'vector', 'result']),
 'environmental': dict_keys(['variety']),
 'unknown': dict_keys(['result'])}

In [53]:
for category in enums['action'].keys():
    for feature in enums['action'][category].keys():
        prefix = 'action.%s.%s'%(category, feature)
        feature_cols = [col for col in vcdb if prefix in col]
        print(prefix, '\n-------------------------')
        print(vcdb[feature_cols].sum(axis=1).value_counts(), '\n')

action.hacking.variety 
-------------------------
0    5755
1    1938
3      94
2      46
dtype: int64 

action.hacking.vector 
-------------------------
0    5755
1    2056
2      21
3       1
dtype: int64 

action.hacking.result 
-------------------------
0    7753
1      48
2      31
3       1
dtype: int64 

action.malware.variety 
-------------------------
0    7200
1     375
9      88
7      74
2      64
3      16
4      14
5       2
dtype: int64 

action.malware.vector 
-------------------------
0    7200
1     621
2      11
3       1
dtype: int64 

action.malware.result 
-------------------------
0    7796
1      29
2       8
dtype: int64 

action.social.variety 
-------------------------
0    7318
1     494
2      20
3       1
dtype: int64 

action.social.vector 
-------------------------
0    7318
1     491
2      23
3       1
dtype: int64 

action.social.target 
-------------------------
0    7318
1     499
2      16
dtype: int64 

action.social.result 
----------------------

### Some tests with hacking variety

In [54]:
(vcdb[[col for col in vcdb if 'action.hacking.variety.' in col]].sum(axis=1) > 1).value_counts()

False    7693
True      140
dtype: int64

In [55]:
vcdb.loc[vcdb[[col for col in vcdb if 'action.hacking.vector.' in col]].sum(axis=1) > 1, [col for col in vcdb if 'action.hacking.vector.' in col]]

,action.hacking.vector.3rd party desktop,action.hacking.vector.Backdoor or C2,action.hacking.vector.Command shell,action.hacking.vector.Desktop sharing,action.hacking.vector.Desktop sharing software,action.hacking.vector.Other,action.hacking.vector.Partner,action.hacking.vector.Physical access,action.hacking.vector.Unknown,action.hacking.vector.VPN,action.hacking.vector.Web application
442,False,False,False,True,True,False,False,False,False,False,False
565,False,False,False,False,False,False,False,False,False,True,True
930,False,False,False,True,True,False,False,False,False,False,False
968,False,False,False,True,True,False,False,False,False,False,False
1980,False,True,True,False,False,False,False,False,False,False,False
2038,False,False,False,False,False,False,False,False,True,False,True
2221,True,False,False,False,True,False,False,False,False,False,False
2361,False,True,False,False,False,False,False,False,False,False,True
3276,True,False,False,False,True,False,False,False,False,False,False
3553,False,False,False,False,False,False,False,False,True,False,True


In [56]:
vcdb.loc[vcdb[[col for col in vcdb if 'action.hacking.variety.' in col]].sum(axis=1) > 1, [col for col in vcdb if 'action.hacking.variety.' in col]]

,action.hacking.variety.Abuse of functionality,action.hacking.variety.Brute force,action.hacking.variety.Buffer overflow,action.hacking.variety.Cache poisoning,action.hacking.variety.Cryptanalysis,action.hacking.variety.CSRF,action.hacking.variety.DoS,action.hacking.variety.Footprinting,action.hacking.variety.Forced browsing,action.hacking.variety.Format string attack,...,action.hacking.variety.Use of backdoor or C2,action.hacking.variety.Use of stolen creds,action.hacking.variety.Virtual machine escape,action.hacking.variety.XML attribute blowup,action.hacking.variety.XML entity expansion,action.hacking.variety.XML external entities,action.hacking.variety.XML injection,action.hacking.variety.XPath injection,action.hacking.variety.XQuery injection,action.hacking.variety.XSS
154,False,True,False,False,False,False,False,False,False,False,...,True,True,False,False,False,False,False,False,False,False
191,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
249,False,False,False,False,True,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
376,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
422,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7493,False,True,False,False,False,False,False,False,False,False,...,True,True,False,False,False,False,False,False,False,False
7537,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,False,False,False,False,True
7682,False,False,False,False,False,False,False,True,False,False,...,True,False,False,False,False,False,False,False,False,False
7750,False,True,False,False,False,False,False,False,False,False,...,True,True,False,False,False,False,False,False,False,False


In [55]:
# Loop through every action type
encoded_action_features = []
ohe_action_features = []


process_action = False

if process_action:
    for action_type, category in tqdm.tqdm(enums['action'].items()):

        # Loop through every categorical feature we wish to create within each action type.
        for feature, values in category.items():
            # Extract the new column name and the one-hot encoded columns we wish to transform to categoricals.
            feature_name = 'action.%s.%s'%(action_type, feature) 
            #print(feature_name)

            feature_cols = [col for col in vcdb if 'action.%s.%s.'%(action_type, feature) in col]

            # Transform to categorical if only one hit
            if (vcdb[feature_cols].sum(axis=1) > 1).any():
                ohe_action_features.append(feature_name)
            else:
                encoded_action_features.append(feature_name)
                # Create new columns
                vcdb[feature_name] = (
                    vcdb[feature_cols]
                        .apply(decode, axis=1)
                        .str.split('.')
                        .str[-1]
                )

                vcdb = vcdb.drop(labels=feature_cols, axis=1)
                assert len([col for col in vcdb if 'action.%s.%s.'%(action_type, feature) in col]) == 0

In [56]:
encoded_action_features

[]

In [57]:
ohe_action_features

[]

In [58]:
for feature in ohe_action_features:
    print(feature+':', (vcdb[[col for col in vcdb if feature+'.' in col]].sum(axis=1) > 1).sum())

## Actor Related Columns

Either we use this as a feature or as a target variable, or we simply drop it.

In [59]:
enums['actor'].keys()

dict_keys(['external', 'internal', 'partner'])

In [60]:
{k: v.keys() for k, v in enums['actor'].items()}

{'external': dict_keys(['variety', 'motive', 'country']),
 'internal': dict_keys(['variety', 'motive', 'job_change']),
 'partner': dict_keys(['motive', 'country'])}

Actor columns don't seem to be particularly useful to us in predicting Impact/Likelihood, although perhaps we can use these as target variables for predicting the actors.

We would achieve this by melting columns where the are multiple actors.

We leave one-hot encoded when using as a feature, and melt for using as a target variable

In [61]:
for category in enums['actor'].keys():
    for feature in enums['actor'][category].keys():
        prefix = 'actor.%s.%s'%(category, feature)
        feature_cols = [col for col in vcdb if prefix in col]
        print(prefix, '\n-------------------------')
        print(vcdb[feature_cols].sum(axis=1).value_counts(), '\n')

actor.external.variety 
-------------------------
0    3846
1    3626
2     356
3       5
dtype: int64 

actor.external.motive 
-------------------------
0    3846
1    3830
2     152
3       5
dtype: int64 

actor.external.country 
-------------------------
0    3846
1    3807
2     161
3      17
5       1
4       1
dtype: int64 

actor.internal.variety 
-------------------------
0    4287
1    3221
2     321
3       4
dtype: int64 

actor.internal.motive 
-------------------------
0    4287
1    3228
2     315
3       3
dtype: int64 

actor.internal.job_change 
-------------------------
0    7547
1     286
dtype: int64 

actor.partner.motive 
-------------------------
0    7484
1     273
2      75
3       1
dtype: int64 

actor.partner.country 
-------------------------
0    7484
1     305
2      44
dtype: int64 



In [62]:
vcdb = vcdb.drop(labels=[col for col in vcdb if col.startswith('actor.')], axis=1)

# Discovery Method

Bad for features, but useful perhaps as target variables

In [63]:
discovery_cols = [col for col in vcdb if 'discovery_method' in col]
discovery_cols

['discovery_method.Ext - actor disclosure',
 'discovery_method.Ext - audit',
 'discovery_method.Ext - customer',
 'discovery_method.Ext - emergency response team',
 'discovery_method.Ext - found documents',
 'discovery_method.Ext - fraud detection',
 'discovery_method.Ext - incident response',
 'discovery_method.Ext - law enforcement',
 'discovery_method.Ext - monitoring service',
 'discovery_method.Ext - other',
 'discovery_method.Ext - suspicious traffic',
 'discovery_method.Ext - unknown',
 'discovery_method.Ext - unrelated 3rd party',
 'discovery_method.Int - antivirus',
 'discovery_method.Int - break in discovered',
 'discovery_method.Int - data loss prevention',
 'discovery_method.Int - financial audit',
 'discovery_method.Int - fraud detection',
 'discovery_method.Int - HIDS',
 'discovery_method.Int - incident response',
 'discovery_method.Int - infrastructure monitoring',
 'discovery_method.Int - IT review',
 'discovery_method.Int - log review',
 'discovery_method.Int - NIDS',


In [64]:
vcdb = vcdb.drop(labels=discovery_cols, axis=1)

## Attribute Related Columns

This section describes which security attributes (of the previously-identified assets) were compromised during the incident. To accomplish this, VERIS uses a paired version of the six primary security attributes of confidentiality/possession, integrity/authenticity, availability/utility. An extension of the “C-I-A Triad,” they are commonly called the “Parkerian Hexad,” after their originator, Donn Parker. Multiple attributes can be affected for any one asset and each attribute contains different metrics.

These are not useful as features for predictions, but may be useful as target variables.

In [65]:
attribute_cols = [col for col in vcdb if col.startswith('attribute.')]
attribute_cols

['attribute.availability.duration.unit.Days',
 'attribute.availability.duration.unit.Hours',
 'attribute.availability.duration.unit.Minutes',
 'attribute.availability.duration.unit.Months',
 'attribute.availability.duration.unit.NA',
 'attribute.availability.duration.unit.Never',
 'attribute.availability.duration.unit.Seconds',
 'attribute.availability.duration.unit.Unknown',
 'attribute.availability.duration.unit.Weeks',
 'attribute.availability.duration.unit.Years',
 'attribute.availability.duration.value',
 'attribute.availability.notes',
 'attribute.availability.variety.Acceleration',
 'attribute.availability.variety.Degradation',
 'attribute.availability.variety.Destruction',
 'attribute.availability.variety.Interruption',
 'attribute.availability.variety.Loss',
 'attribute.availability.variety.Obscuration',
 'attribute.availability.variety.Other',
 'attribute.availability.variety.Unknown',
 'attribute.confidentiality.data_disclosure.No',
 'attribute.confidentiality.data_disclosur

#### Attribute Availability

In [66]:
vcdb[[col for col in vcdb if col.startswith('attribute.availability.duration.unit.')]].sum(axis=1).value_counts()

0    7601
1     232
dtype: int64

In [67]:
vcdb = vcdb.pipe(decode_feature, pattern='attribute.availability.duration.unit.')
vcdb['attribute.availability.duration.unit'].value_counts(dropna=False)

NaN        7601
Never        68
Days         47
Hours        42
Unknown      36
Weeks        14
Months       11
Minutes      11
Years         3
Name: attribute.availability.duration.unit, dtype: int64

In [68]:
vcdb.loc[vcdb['attribute.availability.duration.value'].notnull(),'attribute.availability.duration.value']

98       1.0
133      2.0
237      4.0
259      8.0
374      7.0
        ... 
7108    10.0
7148     3.0
7339    24.0
7537     1.0
7723     3.0
Name: attribute.availability.duration.value, Length: 106, dtype: float64

In [69]:
vcdb[[col for col in vcdb if col.startswith('attribute.availability.variety.')]].sum(axis=1).value_counts()

0    5486
1    2284
2      61
3       2
dtype: int64

#### Attribute Confidentiality

In [70]:
[col for col in vcdb if col.startswith('attribute.confidentiality.')]

['attribute.confidentiality.data_disclosure.No',
 'attribute.confidentiality.data_disclosure.Potentially',
 'attribute.confidentiality.data_disclosure.Unknown',
 'attribute.confidentiality.data_disclosure.Yes',
 'attribute.confidentiality.data_total',
 'attribute.confidentiality.data_victim.Customer',
 'attribute.confidentiality.data_victim.Employee',
 'attribute.confidentiality.data_victim.Other',
 'attribute.confidentiality.data_victim.Partner',
 'attribute.confidentiality.data_victim.Patient',
 'attribute.confidentiality.data_victim.Student',
 'attribute.confidentiality.data_victim.Unknown',
 'attribute.confidentiality.data.amount.Bank',
 'attribute.confidentiality.data.amount.Classified',
 'attribute.confidentiality.data.amount.Copyrighted',
 'attribute.confidentiality.data.amount.Credentials',
 'attribute.confidentiality.data.amount.Digital certificate',
 'attribute.confidentiality.data.amount.Internal',
 'attribute.confidentiality.data.amount.Medical',
 'attribute.confidentiality

In [71]:
vcdb = vcdb.pipe(decode_feature, pattern='attribute.confidentiality.data_disclosure.')
vcdb['attribute.confidentiality.data_disclosure'].value_counts(dropna=False)

Yes            5076
Potentially    1884
NaN             584
Unknown         213
No               76
Name: attribute.confidentiality.data_disclosure, dtype: int64

In [72]:
vcdb[[col for col in vcdb if col.startswith('attribute.confidentiality.data_victim.')]].sum(axis=1).value_counts()

1    4931
0    2829
2      70
3       3
dtype: int64

In [73]:
vcdb[[col for col in vcdb if col.startswith('attribute.confidentiality.data.amount.')]].notnull().any(axis=1).value_counts()

False    4624
True     3209
dtype: int64

In [74]:
vcdb[[col for col in vcdb if col.startswith('attribute.confidentiality.data.variety.')]].sum(axis=1).value_counts()

1    6016
2     859
0     681
3      99
5      91
4      85
7       1
6       1
dtype: int64

#### Attribute Integrity

In [75]:
[col for col in vcdb if col.startswith('attribute.integrity.')]

['attribute.integrity.notes',
 'attribute.integrity.variety.Alter behavior',
 'attribute.integrity.variety.Created account',
 'attribute.integrity.variety.Defacement',
 'attribute.integrity.variety.Fraudulent transaction',
 'attribute.integrity.variety.Hardware tampering',
 'attribute.integrity.variety.Log tampering',
 'attribute.integrity.variety.Misrepresentation',
 'attribute.integrity.variety.Modify configuration',
 'attribute.integrity.variety.Modify data',
 'attribute.integrity.variety.Modify privileges',
 'attribute.integrity.variety.Other',
 'attribute.integrity.variety.Repurpose',
 'attribute.integrity.variety.Software installation',
 'attribute.integrity.variety.Unknown']

In [76]:
vcdb[[col for col in vcdb if col.startswith('attribute.integrity.variety.')]].sum(axis=1).value_counts()

0    5998
1    1251
2     357
3     219
4       7
5       1
dtype: int64

## Targeted Related Columns

This feature might be good as a target variable, but its not very useful to use as a feature.

In [77]:
vcdb = vcdb.pipe(decode_feature, pattern='targeted.')
vcdb['targeted'].value_counts(dropna=False)

NaN              7206
Targeted          178
Opportunistic     176
NA                173
Unknown           100
Name: targeted, dtype: int64

### Corrective Action

In [78]:
vcdb['corrective_action'].isnull().all()

True

In [79]:
cost_corrective_act_cols = [col for col in vcdb if 'cost_corrective_action.' in col]

vcdb[cost_corrective_act_cols].sum(axis=1).value_counts()

0    7828
1       5
dtype: int64

In [80]:
vcdb = vcdb.pipe(decode_feature, pattern='cost_corrective_action.')
vcdb['cost_corrective_action'].value_counts(dropna=False)

NaN                     7828
Unknown                    2
Something in-between       2
Simple and cheap           1
Name: cost_corrective_action, dtype: int64

## Asset Related Columns

Either we use this as a feature or as a target variable, or we simply drop it.

In [81]:
enums['asset'].keys()

dict_keys(['assets', 'ownership', 'cloud', 'hosting', 'management', 'role', 'country'])

In [82]:
set([col[:col.rindex('.')] for col in vcdb if col.startswith('asset.') if col.count('.') > 1])

{'asset.accessibility',
 'asset.assets.amount',
 'asset.assets.variety',
 'asset.cloud',
 'asset.country',
 'asset.governance',
 'asset.hosting',
 'asset.management',
 'asset.ownership',
 'asset.variety'}

### asset.cloud

In [83]:
asset_cloud_cols = [col for col in vcdb if col.startswith('asset.cloud.')]
asset_cloud_cols

['asset.cloud.Customer attack',
 'asset.cloud.Hosting error',
 'asset.cloud.Hosting governance',
 'asset.cloud.Hypervisor',
 'asset.cloud.NA',
 'asset.cloud.No',
 'asset.cloud.Other',
 'asset.cloud.Partner application',
 'asset.cloud.Unknown',
 'asset.cloud.User breakout']

In [84]:
vcdb[asset_cloud_cols].sum(axis=1).value_counts()

0    7733
1     100
dtype: int64

In [85]:
vcdb = vcdb.pipe(decode_feature, pattern='asset.cloud.')
vcdb['asset.cloud'].value_counts(dropna=False)

NaN                    7733
Unknown                  69
NA                       12
Hosting error             7
Other                     5
Hosting governance        3
Customer attack           2
Partner application       2
Name: asset.cloud, dtype: int64

###  asset.accessibility

ASSET.ACCESSIBILITY
External: Publicly accessible

Internal: Internally accessible

Isolated: Internally isolated or restricted environment

Unknown: Unknown

NA: Not applicable


http://veriscommunity.net/enums.html#section-asset

In [86]:
asset_accessibility_cols = [col for col in vcdb if 'asset.accessibility.' in col]
vcdb[asset_accessibility_cols].sum(axis=1).value_counts()

0    7821
1      12
dtype: int64

In [87]:
vcdb = vcdb.pipe(decode_feature, pattern='asset.accessibility.')
vcdb['asset.accessibility'].value_counts(dropna=False)

NaN         7821
External      11
Unknown        1
Name: asset.accessibility, dtype: int64

###  asset.assets
http://veriscommunity.net/enums.html#section-asset

### Asset Variety
These are multi-select and so we leave them one-hot encoded.

In [88]:
asset_variety_cols = [col for col in vcdb if 'asset.assets.variety.' in col]
vcdb.loc[vcdb[asset_variety_cols].sum(axis=1) > 1, asset_variety_cols]

,asset.assets.variety.E - Other,asset.assets.variety.E - Telematics,asset.assets.variety.E - Telemetry,asset.assets.variety.E - Unknown,asset.assets.variety.M - Disk drive,asset.assets.variety.M - Disk media,asset.assets.variety.M - Documents,asset.assets.variety.M - Fax,asset.assets.variety.M - Flash drive,asset.assets.variety.M - Other,...,asset.assets.variety.U - Media,asset.assets.variety.U - Mobile phone,asset.assets.variety.U - Other,asset.assets.variety.U - Peripheral,asset.assets.variety.U - POS terminal,asset.assets.variety.U - Tablet,asset.assets.variety.U - Telephone,asset.assets.variety.U - Unknown,asset.assets.variety.U - VoIP phone,asset.assets.variety.Unknown
16,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
18,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
22,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
36,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
48,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7781,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7783,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7793,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7794,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [89]:
vcdb[asset_variety_cols].sum(axis=1).value_counts()

1     6913
2      633
3      158
5       98
4       29
7        1
10       1
dtype: int64

### Asset Amounts
These are multi-select and so we leave them one-hot encoded.

In [90]:
asset_amt_cols =[col for col in vcdb if 'asset.assets.amount.' in col]
vcdb.loc[vcdb[asset_amt_cols].notnull().any(axis=1) & 
         (vcdb[asset_amt_cols]>1).any(axis=1), 
         asset_amt_cols]

,asset.assets.amount.E - Other,asset.assets.amount.E - Telematics,asset.assets.amount.E - Telemetry,asset.assets.amount.E - Unknown,asset.assets.amount.M - Disk drive,asset.assets.amount.M - Disk media,asset.assets.amount.M - Documents,asset.assets.amount.M - Fax,asset.assets.amount.M - Flash drive,asset.assets.amount.M - Other,...,asset.assets.amount.U - Media,asset.assets.amount.U - Mobile phone,asset.assets.amount.U - Other,asset.assets.amount.U - Peripheral,asset.assets.amount.U - POS terminal,asset.assets.amount.U - Tablet,asset.assets.amount.U - Telephone,asset.assets.amount.U - Unknown,asset.assets.amount.U - VoIP phone,asset.assets.amount.Unknown
197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
281,NaN,NaN,NaN,NaN,NaN,NaN,1100.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
471,NaN,NaN,NaN,NaN,NaN,NaN,703.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
481,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
544,NaN,NaN,NaN,NaN,NaN,NaN,1020.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7479,NaN,NaN,NaN,NaN,NaN,NaN,10500.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Asset Country

In [91]:
vcdb[[col for col in vcdb if col.startswith('asset.country.')]].sum(axis=1).value_counts()

0    7735
1      98
dtype: int64

In [92]:
vcdb = vcdb.pipe(decode_feature, pattern='asset.country.')
vcdb['asset.country'].value_counts(dropna=False)

NaN        7735
US           66
GB            8
CA            8
Unknown       6
AU            3
MT            1
HK            1
JP            1
AF            1
GR            1
CN            1
ES            1
Name: asset.country, dtype: int64

### Asset Governance
These appear to be multi-select and so we leave as one-hot encoded

In [93]:
[col for col in vcdb if col.startswith('asset.governance.')]

['asset.governance.3rd party hosted',
 'asset.governance.3rd party managed',
 'asset.governance.3rd party owned',
 'asset.governance.Internally isolated',
 'asset.governance.Other',
 'asset.governance.Personally owned',
 'asset.governance.Unknown',
 'asset.governance.Victim governed']

In [94]:
vcdb[[col for col in vcdb if col.startswith('asset.governance.')]].sum(axis=1).value_counts()

0    6899
1     752
2     122
3      60
dtype: int64

### Asset Hosting

In [95]:
[col for col in vcdb if col.startswith('asset.hosting.')]

['asset.hosting.External',
 'asset.hosting.External dedicated',
 'asset.hosting.External shared',
 'asset.hosting.Internal',
 'asset.hosting.NA',
 'asset.hosting.Other',
 'asset.hosting.Unknown']

In [96]:
vcdb[[col for col in vcdb if col.startswith('asset.hosting.')]].sum(axis=1).value_counts()

0    7833
dtype: int64

In [97]:
vcdb = vcdb.pipe(decode_feature, pattern='asset.hosting.')
vcdb['asset.hosting'].value_counts(dropna=False)

NaN    7833
Name: asset.hosting, dtype: int64

### Asset Hosting

In [98]:
[col for col in vcdb if col.startswith('asset.management.')]

['asset.management.External',
 'asset.management.Internal',
 'asset.management.NA',
 'asset.management.Other',
 'asset.management.Unknown']

In [99]:
vcdb[[col for col in vcdb if col.startswith('asset.management.')]].sum(axis=1).value_counts()

0    7833
dtype: int64

In [100]:
vcdb= vcdb.pipe(decode_feature, pattern='asset.management.')
vcdb['asset.management'].value_counts(dropna=False)

NaN    7833
Name: asset.management, dtype: int64

### Asset Variety
Is multiselect so we leave as is

In [101]:
vcdb[[col for col in vcdb if col.startswith('asset.variety.')]].sum(axis=1).value_counts()

1    7139
2     468
3     123
4      93
0       6
5       4
dtype: int64

In [102]:
vcdb[[col for col in vcdb if col.startswith('asset.variety.')]]

,asset.variety.Server,asset.variety.Network,asset.variety.User Dev,asset.variety.Media,asset.variety.Person,asset.variety.Kiosk/Term,asset.variety.Unknown,asset.variety.Embedded
0,True,False,False,False,False,False,False,False
1,False,False,True,False,False,False,False,False
2,False,False,False,True,False,False,False,False
3,True,False,False,False,False,False,False,False
4,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...
7828,True,False,False,False,False,False,False,False
7829,False,False,True,False,False,False,False,False
7830,False,False,False,True,False,False,False,False
7831,False,False,True,False,False,False,False,False


In [103]:
pd.Series(vcdb.columns).to_clipboard()

# Pattern features
There potentially some good useful information here.
These pattern features aren't very useful for predictions, but may be good for 

In [104]:
vcdb['pattern'].value_counts(dropna=False)

Miscellaneous Errors      1812
Privilege Misuse          1597
Lost and Stolen Assets    1460
Everything Else           1026
Web Applications           896
Payment Card Skimmers      278
Crimeware                  267
Cyber-Espionage            247
Denial of Service          162
Point of Sale               88
Name: pattern, dtype: int64

The other pattern features appear to be multiselect

In [105]:
[col for col in vcdb if col.startswith('pattern.')]

['pattern.Point of Sale',
 'pattern.Web Applications',
 'pattern.Privilege Misuse',
 'pattern.Lost and Stolen Assets',
 'pattern.Miscellaneous Errors',
 'pattern.Crimeware',
 'pattern.Payment Card Skimmers',
 'pattern.Denial of Service',
 'pattern.Cyber-Espionage',
 'pattern.Everything Else']

We leave as one hot encoded because some have both

In [106]:
vcdb[[col for col in vcdb if col.startswith('pattern.')]].sum(axis=1).value_counts(dropna=False)

1    7732
2     101
dtype: int64

In [107]:
vcdb.loc[vcdb[[col for col in vcdb if col.startswith('pattern.')]].sum(axis=1)>1, [col for col in vcdb if col.startswith('pattern.')]]

,pattern.Point of Sale,pattern.Web Applications,pattern.Privilege Misuse,pattern.Lost and Stolen Assets,pattern.Miscellaneous Errors,pattern.Crimeware,pattern.Payment Card Skimmers,pattern.Denial of Service,pattern.Cyber-Espionage,pattern.Everything Else
16,False,False,True,False,False,False,False,False,True,False
375,False,False,True,True,False,False,False,False,False,False
634,False,False,True,True,False,False,False,False,False,False
957,False,False,True,False,False,False,True,False,False,False
1026,False,False,True,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
7320,False,True,True,False,False,False,False,False,False,False
7358,False,False,False,True,False,False,True,False,False,False
7450,False,False,False,False,False,False,False,True,True,False
7593,False,False,True,True,False,False,False,False,False,False


# Plus features
There potentially some good shit here

In [108]:
plus_cols = [col for col in vcdb if 'plus' in col]
plus_cols

['plus.analysis_status.Finalized',
 'plus.analysis_status.First pass',
 'plus.analysis_status.In-progress',
 'plus.analysis_status.Ineligible',
 'plus.analysis_status.Needs review',
 'plus.analysis_status.Validated',
 'plus.analyst',
 'plus.analyst_notes',
 'plus.antiforensic_measures.Data Corruption',
 'plus.antiforensic_measures.Data Hiding',
 'plus.antiforensic_measures.Data Wiping',
 'plus.antiforensic_measures.No evidence of AF',
 'plus.antiforensic_measures.Other',
 'plus.antiforensic_measures.Unknown',
 'plus.asset_os.Android',
 'plus.asset_os.Apple iOS',
 'plus.asset_os.BlackBerry OS',
 'plus.asset_os.Linux',
 'plus.asset_os.Mac OSX',
 'plus.asset_os.Mainframe',
 'plus.asset_os.Not applicable',
 'plus.asset_os.Other',
 'plus.asset_os.Symbian',
 'plus.asset_os.Unix',
 'plus.asset_os.Unknown',
 'plus.asset_os.webOS',
 'plus.asset_os.Windows',
 'plus.asset_os.Windows Phone',
 'plus.asset.total',
 'plus.attack_difficulty_initial.High',
 'plus.attack_difficulty_initial.Low',
 'plus.

In [109]:
set([col[:col.rindex('.')] for col in plus_cols if col.count('.')>1 and 'notes' not in col])

{'plus.analysis_status',
 'plus.antiforensic_measures',
 'plus.asset',
 'plus.asset_os',
 'plus.attack_difficulty_initial',
 'plus.attack_difficulty_legacy',
 'plus.attack_difficulty_subsequent',
 'plus.attribute.confidentiality',
 'plus.event_chain',
 'plus.event_chain.action',
 'plus.event_chain.actor',
 'plus.event_chain.asset',
 'plus.event_chain.attribute',
 'plus.pci',
 'plus.pci.compliance_status',
 'plus.pci.merchant_level',
 'plus.pci.req_1',
 'plus.pci.req_10',
 'plus.pci.req_11',
 'plus.pci.req_12',
 'plus.pci.req_2',
 'plus.pci.req_3',
 'plus.pci.req_4',
 'plus.pci.req_5',
 'plus.pci.req_6',
 'plus.pci.req_7',
 'plus.pci.req_8',
 'plus.pci.req_9',
 'plus.security_maturity',
 'plus.unfollowed_policies',
 'plus.unknown_unknowns'}

In [110]:
vcdb[plus_cols].isnull().all().value_counts()

False    153
True       7
dtype: int64

In [111]:
vcdb[plus_cols].notnull().all().value_counts()

True     142
False     18
dtype: int64

#### Plus Analysis Status

In [112]:
vcdb = vcdb.pipe(decode_feature, pattern='plus.analysis_status.')
vcdb['plus.analysis_status'].value_counts(dropna=False)

First pass      4858
Finalized       1808
Validated        832
NaN              281
Needs review      50
Ineligible         4
Name: plus.analysis_status, dtype: int64

#### Plus Analysis Status

In [113]:
vcdb = vcdb.pipe(decode_feature, pattern='plus.antiforensic_measures.')
vcdb['plus.antiforensic_measures'].value_counts(dropna=False)

NaN    7833
Name: plus.antiforensic_measures, dtype: int64

In [114]:
vcdb[[col for col in vcdb if col.startswith('plus.asset_os.')]].sum(axis=1).value_counts()

0    7819
1      14
dtype: int64

#### Plus Attack Difficulty

In [115]:
vcdb = vcdb.pipe(decode_feature, pattern='plus.attack_difficulty_initial.')
vcdb['plus.attack_difficulty_initial'].value_counts(dropna=False)

NaN               7672
Low                106
Very Low            22
Not Applicable      18
Unknown             11
Moderate             4
Name: plus.attack_difficulty_initial, dtype: int64

In [116]:
vcdb = vcdb.pipe(decode_feature, pattern='plus.attack_difficulty_legacy.')
vcdb['plus.attack_difficulty_legacy'].value_counts(dropna=False)

NaN               7780
Low                 34
Not Applicable       7
Very Low             5
Moderate             3
Unknown              3
High                 1
Name: plus.attack_difficulty_legacy, dtype: int64

In [117]:
vcdb = vcdb.pipe(decode_feature, pattern='plus.attack_difficulty_subsequent.')
vcdb['plus.attack_difficulty_subsequent'].value_counts(dropna=False)

NaN               7726
Low                 69
Not Applicable      19
Unknown              9
Very Low             8
Moderate             2
Name: plus.attack_difficulty_subsequent, dtype: int64

#### Plus Attribute

In [118]:
[col for col in vcdb if col.startswith('plus.attribute.confidentiality.')]

['plus.attribute.confidentiality.credit_monitoring',
 'plus.attribute.confidentiality.data_abuse',
 'plus.attribute.confidentiality.data_misuse',
 'plus.attribute.confidentiality.partner_data',
 'plus.attribute.confidentiality.partner_number']

#### Plus Event Chain

In [119]:
vcdb = vcdb.pipe(decode_feature, pattern='plus.event_chain.action.')
vcdb['plus.event_chain.action'].value_counts(dropna=False)

NaN    7833
Name: plus.event_chain.action, dtype: int64

In [120]:
vcdb = vcdb.pipe(decode_feature, pattern='plus.event_chain.asset.')
vcdb['plus.event_chain.asset'].value_counts(dropna=False)

NaN    7833
Name: plus.event_chain.asset, dtype: int64

In [121]:
vcdb = vcdb.pipe(decode_feature, pattern='plus.event_chain.attribute.')
vcdb['plus.event_chain.attribute'].value_counts(dropna=False)

NaN    7833
Name: plus.event_chain.attribute, dtype: int64

In [122]:
vcdb[[col for col in vcdb if col.startswith('plus.asset_os.')]].sum(axis=1).value_counts()

0    7819
1      14
dtype: int64

#### Plus PCI reqs

In [123]:
pci_reqs = set([col[:col.rindex('.')] for col in vcdb if col.startswith('plus.pci.req_')])

In [124]:
for req in tqdm.tqdm(pci_reqs):
    vcdb = vcdb.pipe(decode_feature, pattern=req+'.')

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:08<00:00,  1.35it/s]


In [125]:
vcdb[pci_reqs].isnull().all().all()

True

#### Plus PCI Security Maturity

In [126]:
vcdb = vcdb.pipe(decode_feature, pattern='plus.security_maturity.')
vcdb['plus.security_maturity'].value_counts(dropna=False)

NaN    7833
Name: plus.security_maturity, dtype: int64

#### Plus PCI Unfollowed Policies

In [127]:
vcdb = vcdb.pipe(decode_feature, pattern='plus.unfollowed_policies.')
vcdb['plus.unfollowed_policies'].value_counts(dropna=False)

NaN    7833
Name: plus.unfollowed_policies, dtype: int64

#### Plus PCI Unknown unknowns

In [128]:
vcdb = vcdb.pipe(decode_feature, pattern='plus.unknown_unknowns.')
vcdb['plus.unknown_unknowns'].value_counts(dropna=False)

NaN                                                                 7827
Assets that had unknown network connections or accessibility           3
Data the organization did not know existed on a particular asset       2
Unknown                                                                1
Name: plus.unknown_unknowns, dtype: int64

## Write decoded dataset to file

In [129]:
vcdb.to_excel('data/vcdb_decoded.xlsx', index=False)

C:\Users\josep\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:937: UserWarning: Ignoring URL 'http://www.katc.com/news/data-breach-may-have-exposed-jp-morgan-chase-cardholders/;http://www.bayoubuzz.com/louisiana-news/baton-rouge-news/item/561503-hackers-may-have-exposed-personal-info-of-la-residents;http://www.huffingtonpost.com/2013/12/05/jpmorgan-cyber-attack_n_4388779.html;http://arstechnica.com/security/2013/12/hack-on-jpmorgan-website-exposes-data-for-465000-card-holders/;http://www.databreaches.net/louisiana-agencies-notified-of-breach-across-certain-jpmorgan-chase-systems-breach-not-confined-to-louisiana/;http://www.databreaches.net/jpmorgan-chase-breach-update-pile-on/;http://www.nbcphiladelphia.com/news/business/Data-Breach-to-Affect-Pa-Cardholders-234676751.html;http://www.sltrib.com/sltrib/news/57229169-78/breach-jpmorgan-customers-data.html.csp;http://www.hispanicbusiness.com/2013/12/6/data_breach_exposes_account_information_for.htm;http://www.bostonglobe.com/metro/201

In [130]:
vcdb.dtypes.value_counts()

bool       476
float64    129
object      86
int64        1
dtype: int64

In [132]:
vcdb.select_dtypes(['float64', 'object', 'int64']).to_excel('data/vcdb_nobools.xlsx', index=False)

FileCreateError: [Errno 13] Permission denied: 'data/vcdb_nobools.xlsx'

In [ ]:
vcdb.select_dtypes('O').nunique().sort_values(ascending=False).to_clipboard()

In [ ]:
vcdb.apply(lambda col: col.astype(str).str.contains('http')).sum().sort_values()

In [ ]:
vcdb['reference']